In [ ]:
import pandas as pd
from collections import Counter
import json

def load_item_profiles(json_path):
    """JSON 파일에서 item_profiles를 불러옵니다."""
    with open(json_path, 'r', encoding='utf-8') as f:
        item_profiles = json.load(f)
    return item_profiles

def get_customer_info(customers_df, customer_id):
    """고객 ID에 대한 성별과 연령대 정보를 반환합니다."""
    customer_info = customers_df[customers_df['고객식별ID'] == customer_id]
    if customer_info.empty:
        return None
    return customer_info.iloc[0]['성별'], customer_info.iloc[0]['연령대']

def find_most_frequent_items_by_demographics(orders_paths, customers_df, item_profiles, gender, age_group, number_filter, top_n=10):
    """성별과 연령대별로 가장 많이 주문된 상품을 찾고, 상품 이름이 특정 숫자로 끝나는 것만 필터링."""
    demographic_item_count = Counter()
    
    for orders_path in orders_paths:
        df = pd.read_csv(orders_path)
        
        # 고객 데이터를 주문 데이터와 병합하여 성별 및 연령대 필터링
        merged_df = pd.merge(df, customers_df, on='고객식별ID')
        filtered_df = merged_df[(merged_df['성별'] == gender) & (merged_df['연령대'] == age_group)]
        
        # 해당 성별 및 연령대의 상품 주문 수를 계산
        for item_id in filtered_df['상품ID']:
            if item_id in item_profiles:
                demographic_item_count[item_id] += 1
    
    # 상위 top_n개의 상품을 선택
    most_frequent_items = demographic_item_count.most_common()
    
    # 상품 이름이 특정 숫자로 끝나는지 확인하며 필터링
    recommendations = []
    for item_id, count in most_frequent_items:
        item_name = item_profiles[item_id].get('name', 'Unknown')
        if item_name.endswith(str(number_filter)):  # 숫자 필터링
            item_categories = item_profiles[item_id].get('categories', [])
            category_path = ' > '.join(item_categories) if item_categories else 'Unknown'
            
            recommendations.append({
                'item_id': item_id,
                'name': item_name,
                'categories': category_path,
                'order_count': count
            })
        
        # top_n 개수만큼 결과 제한
        if len(recommendations) >= top_n:
            break
    
    return recommendations

def main():
    # 데이터 파일 경로
    item_profiles_path = 'item_profiles_with_embeddings.json'
    customers_path = 'customers_df.csv'  # 고객 데이터 CSV 파일
    orders_paths = ['order1_df.csv', 'order2_df.csv', 'order3_df.csv', 'order4_df.csv', 'order5_df.csv']
    
    # 데이터 로드
    item_profiles = load_item_profiles(item_profiles_path)
    customers_df = pd.read_csv(customers_path)
    
    # 사용자 입력
    customer_id = input("Enter the customer ID: ")
    customer_info = get_customer_info(customers_df, customer_id)
    
    if not customer_info:
        print("Customer ID not found.")
        return
    
    gender, age_group = customer_info
    print(f"Customer {customer_id} is {gender}, {age_group} age group.")
    
    # 숫자 필터 입력
    number_filter = input("Enter a number (1-5) to filter items by name ending: ")
    if not number_filter.isdigit() or int(number_filter) not in range(1, 6):
        print("Invalid number filter. Please enter a number between 1 and 5.")
        return
    number_filter = int(number_filter)
    
    # 추천 상품 찾기
    recommendations = find_most_frequent_items_by_demographics(
        orders_paths, customers_df, item_profiles, gender, age_group, number_filter
    )
    
    # 추천 결과 출력
    print("\nRecommended items based on gender, age group, and name ending with number:")
    for rec in recommendations:
        print(f"상품 ID: {rec['item_id']}, 이름: {rec['name']}, 카테고리: {rec['categories']}, 주문 횟수: {rec['order_count']}")

if __name__ == "__main__":
    main()